In [0]:
#  !wget http://files.fast.ai/data/dogscats.zip

In [0]:
# !unzip dogscats.zip

In [0]:
import tensorflow as tf
import threading, random, sys, os
from datetime import datetime
import numpy as np
import six

In [0]:
NUM_THREADS = 2 #@param {type:"integer"}
OUTPUT_DIR = './' #@param {type:"string"}
TRAIN_DIR = './dogscats/train' #@param {type:"string"}
VALID_DIR = './dogscats/valid' #@param {type:"string"}
NUM_TRAIN_SHARDS = 2 #@param {type:"integer"}
NUM_VALID_SHARDS = 2 #@param {type:"integer"}
JPEG_FILE_EXT = '.jpg' #@param {type:"string"}


In [0]:
from google.colab import auth
auth.authenticate_user()

In [0]:
def _int64_feature(value):
  """Wrapper for inserting int64 features into Example proto."""
  if not isinstance(value, list):
    value = [value]
  return tf.train.Feature(int64_list=tf.train.Int64List(value=value))

In [0]:
def _bytes_feature(value):
  """Wrapper for inserting bytes features into Example proto."""
  if six.PY3 and isinstance(value, six.text_type):           
    value = six.binary_type(value, encoding='utf-8') 
  return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

In [0]:
def _process_image_files_batch(thread_index, ranges, name, filenames, labels, num_shards):
  num_threads = len(ranges)
  assert not num_shards % num_threads
  num_shards_per_batch = num_shards // num_threads

  shard_ranges = np.linspace(ranges[thread_index][0], ranges[thread_index][1], num_shards_per_batch + 1).astype(int)
  num_files_in_thread = ranges[thread_index][1] - ranges[thread_index][0]

  counter = 0
  for s in range(num_shards_per_batch):
    # Generate a sharded version of the file name, e.g. 'train-00002-of-00010'
    shard = thread_index * num_shards_per_batch + s
    output_filename = '%s-%.5d-of-%.5d' % (name, shard, num_shards)
    output_file = os.path.join(OUTPUT_DIR, output_filename)
    writer = tf.python_io.TFRecordWriter(output_file)

    shard_counter = 0
    files_in_shard = np.arange(shard_ranges[s], shard_ranges[s + 1], dtype=int)

    for i in files_in_shard:
      filename = filenames[i]
      label = labels[i]

      with tf.gfile.GFile(filename, 'rb') as f:
        image_buffer = f.read()
        example = tf.train.Example(features=tf.train.Features(feature={
          'image/class/label': _int64_feature(label),
          'image/filename': _bytes_feature(os.path.basename(filename)),
          'image/encoded': _bytes_feature(image_buffer)}))
        writer.write(example.SerializeToString())
        
      shard_counter += 1
      counter += 1

      if not counter % 1000:
        print('%s [thread %d]: Processed %d of %d images in thread batch.' %
              (datetime.now(), thread_index, counter, num_files_in_thread))
        sys.stdout.flush()

    writer.close()
    print('%s [thread %d]: Wrote %d images to %s' %
          (datetime.now(), thread_index, shard_counter, output_file))
    sys.stdout.flush()
    shard_counter = 0
  print('%s [thread %d]: Wrote %d images to %d shards.' %
        (datetime.now(), thread_index, counter, num_files_in_thread))
  sys.stdout.flush()

In [0]:
def _process_image_files(name, filenames, labels, num_shards):
  """Process and save list of images as TFRecord of Example protos.
  Args:
    name: string, unique identifier specifying the data set
    filenames: list of strings; each string is a path to an image file
    labels: list of integer; each integer identifies the ground truth
    num_shards: integer number of shards for this data set.
  """

  spacing = np.linspace(0, len(filenames), NUM_THREADS + 1).astype(np.int)
  ranges = []
  threads = []
  for i in range(len(spacing) - 1):
    ranges.append([spacing[i], spacing[i + 1]])

  print('Launching %d threads for spacings: %s' % (NUM_THREADS, ranges))
  sys.stdout.flush()

  coord = tf.train.Coordinator()

  threads = []
  for thread_index in range(len(ranges)):
    args = (thread_index, ranges, name, filenames, labels, num_shards)
    t = threading.Thread(target=_process_image_files_batch, args=args)
    t.start()
    threads.append(t)

  coord.join(threads)
  
  print('%s: Finished writing all %d images in data set.' % (datetime.now(), len(filenames)))
  sys.stdout.flush()


In [0]:
def _find_image_files(data_dir):
  challenge_synsets = ['cats', 'dogs']

  labels = []
  filenames = []

  label_index = 0

  for synset in challenge_synsets:
    jpeg_file_path = f'{data_dir}/{synset}/*{JPEG_FILE_EXT}'
    matching_files = tf.gfile.Glob(jpeg_file_path)
    labels.extend([label_index] * len(matching_files))
    filenames.extend(matching_files)
    label_index += 1

  shuffled_index = list(range(len(filenames)))
  random.seed(12345)
  random.shuffle(shuffled_index)

  filenames = [filenames[i] for i in shuffled_index]
  labels = [labels[i] for i in shuffled_index]

  print('Found %d JPEG files across %d labels inside %s.' % (len(filenames), len(challenge_synsets), data_dir))
  return filenames, labels

In [0]:
def _process_dataset(name, directory, num_shards):
  """Process a complete data set and save it as a TFRecord.
  Args:
    name: string, unique identifier specifying the data set.
    directory: string, root path to the data set.
    num_shards: integer number of shards for this data set.
  """
  filenames, labels = _find_image_files(directory)
  _process_image_files(name, filenames, labels, num_shards)

In [55]:
_process_dataset('valid', VALID_DIR, NUM_VALID_SHARDS)

Found 2000 JPEG files across 2 labels inside ./dogscats/valid.
Launching 2 threads for spacings: [[0, 1000], [1000, 2000]]
2019-03-21 06:10:01.161528 [thread 1]: Processed 1000 of 1000 images in thread batch.
2019-03-21 06:10:01.163761 [thread 1]: Wrote 1000 images to ./valid-00001-of-00002
2019-03-21 06:10:01.165745 [thread 1]: Wrote 1000 images to 1000 shards.
2019-03-21 06:10:01.177368 [thread 0]: Processed 1000 of 1000 images in thread batch.
2019-03-21 06:10:01.181374 [thread 0]: Wrote 1000 images to ./valid-00000-of-00002
2019-03-21 06:10:01.183504 [thread 0]: Wrote 1000 images to 1000 shards.
2019-03-21 06:10:01.449832: Finished writing all 2000 images in data set.


In [41]:
_process_dataset('train', TRAIN_DIR, NUM_TRAIN_SHARDS)

Found 23000 JPEG files across 2 labels inside ./dogscats/train.
Launching 2 threads for spacings: [[0, 11500], [11500, 23000]]
2019-03-21 05:56:14.393072 [thread 1]: Processed 1000 of 11500 images in thread batch.
2019-03-21 05:56:14.420148 [thread 0]: Processed 1000 of 11500 images in thread batch.
2019-03-21 05:56:15.204296 [thread 1]: Processed 2000 of 11500 images in thread batch.
2019-03-21 05:56:15.232454 [thread 0]: Processed 2000 of 11500 images in thread batch.
2019-03-21 05:56:15.920541 [thread 1]: Processed 3000 of 11500 images in thread batch.
2019-03-21 05:56:16.063171 [thread 0]: Processed 3000 of 11500 images in thread batch.
2019-03-21 05:56:16.580462 [thread 1]: Processed 4000 of 11500 images in thread batch.
2019-03-21 05:56:16.746734 [thread 0]: Processed 4000 of 11500 images in thread batch.
2019-03-21 05:56:17.237826 [thread 1]: Processed 5000 of 11500 images in thread batch.
2019-03-21 05:56:17.400392 [thread 0]: Processed 5000 of 11500 images in thread batch.
201

In [42]:
ls -l

total 1398440
drwxrwxr-x 7 root root      4096 Oct  9  2016 dogscats/
-rw-r--r-- 1 root root 857214334 Apr  1  2017 dogscats.zip
drwxr-xr-x 1 root root      4096 Mar  8 17:26 sample_data/
-rw-r--r-- 1 root root 264165487 Mar 21 05:56 train-00000-of-00002
-rw-r--r-- 1 root root 264854972 Mar 21 05:56 train-00001-of-00002
-rw-r--r-- 1 root root  22848967 Mar 21 05:55 validation-00000-of-00002
-rw-r--r-- 1 root root  22889488 Mar 21 05:55 validation-00001-of-00002


In [46]:
!gsutil cp train* gs://gs_colab/dvc_tfrec

Copying file://train-00000-of-00002 [Content-Type=application/octet-stream]...
==> NOTE: You are uploading one or more large file(s), which would run
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, computing checksums on composite objects is
so slow that gsutil disables downloads of composite objects.

Copying file://train-00001-of-00002 [Content-Type=application/octet-stream]...
/
Operation completed over 2 objects/504.5 MiB.                                    


In [56]:
!gsutil cp valid* gs://gs_colab/dvc_tfrec

Copying file://valid-00000-of-00002 [Content-Type=application/octet-stream]...
Copying file://valid-00001-of-00002 [Content-Type=application/octet-stream]...
\
Operation completed over 2 objects/43.6 MiB.                                     
